In [ ]:
import requests
from IPython.display import JSON

CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"

## 1. Sign in
Retrieve access token from Keycloak

In [ ]:
# Login to Keycloak
CLIENT_ID = "lakekeeper-operator"
CLIENT_SECRET = "bvMiebZ23atcNaE703kKWb2zTQSl1mGE"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "lakekeeper"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)

access_token = response.json()['access_token']

## 2. Check lakekeeper infos

When lakekeeper was bootstrapped we added few permissions. <br/>
Management REST API Swagger available: http://localhost:8181/swagger-ui/#/


In [ ]:
# Server info

response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/info",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
# Who am I

response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/whoami",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
## List users
## Users must be registered in Keycloak AND connect once in lakekeeper UI.

response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/user",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
## List server permissions (eg admin, operator)
## See https://docs.lakekeeper.io/docs/nightly/authorization/

response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
)
JSON(response.json())

In [ ]:
## List (default) project permissions (eg project_admin, data_admin, select etc.)

response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/permissions/project/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
)
JSON(response.json())

### 3. Assign additional permissions

We can assign permission to a lakekeeper at different levels, propagated to object children:
1. Server
2. Project
3. Namespace (and optional) subnamespace
4. Table

Eg. if a user has "create" permission at project level he can create any object within all namespaces of the project.

In [ ]:
def get_user_id(user_name: str) -> str:
    response = requests.get(
        url=f"{MANAGEMENT_URL}/v1/user",
        headers={"Authorization": f"Bearer {access_token}"},
    )
    response.raise_for_status()
    return (u["id"] for u in response.json()["users"] if u["name"] == user_name).__next__()

In [ ]:
user_id = get_user_id("Peter Cold")
user_id

#### 1. Server role

In [ ]:
permissions_add = ["operator"] # or admin

In [ ]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/permissions/server/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
    json={
        "writes": [
            {
                "user": user_id,
                "type": p
            }
            for p in permissions_add
        ]
    }
)
response.text

#### 2. Project role

In [ ]:
# Permissions for project are: 
# "project_admin", "security_admin", "data_admin", "manage_grants", "pass_grants", etc.
permissions_add = ["data_admin"]

In [ ]:
user_id = get_user_id("chief data")
user_id

In [ ]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/permissions/project/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
    json={        
        "writes": [
            {
                "type": p,
                "user": user_id
            }
            for p in permissions_add
        ]
    }
)
response.text

#### Need to get warehouse id to manipulate project's children objects

In [ ]:
response = requests.get(
    f"{MANAGEMENT_URL}/v1/warehouse",
    headers={"Authorization": f"Bearer {access_token}"},
)
warehouse_id = response.json()["warehouses"][0]["id"]  # Assuming only one warehouse
print(f"Warehouse id: {warehouse_id}")

#### 3. Namespace roles

Need to retrieve namespace uuid. Namespaces can be identified by array because API handle sub namespaces. For example

```
└── namespace1
    ├── subns1
    └── subns2
        └── subsubns1
```

* `namespace1` is identified by name with array `["namespace1"]`
* `namespace1.subns1` is identified by name with array `["namespace1", "subns1"]`
* `namespace1.subns2.subsubns1` is identified by name with array `["namespace1", "subns2", "subsubns1"]`

----
List namespace using `parent` query parameter:

Listing root namespaces (no subnamespace) without parent: `{CATALOG_URL}/v1/{warehouse_id}/namespaces`
<br/>Returns namespace1, namespace2

Listing 1st level subnamespaces: `{CATALOG_URL}/v1/{warehouse_id}/namespaces?parent=namespace1`
<br/> Returns subns1, subns2

Listing deeper levels subnamespaces by adding `%1F` (or `\x1f` in decoded version) seperator: `{CATALOG_URL}/v1/{warehouse_id}/namespaces?parent=namespace1%1Fsubns2`
<br/> Returns subsubns1

In [ ]:
# Example of sub namespace
response = requests.get(
    f"{CATALOG_URL}/v1/{warehouse_id}/namespaces",
    params={"returnUuids": "true", "parent": "finance"},
    headers={"Authorization": f"Bearer {access_token}"},
)
response.text

In [ ]:
# Example of sub sub namespace
response = requests.get(
    f"{CATALOG_URL}/v1/{warehouse_id}/namespaces",
    params={"returnUuids": "true", "parent": "finance\x1frevenue"},
    headers={"Authorization": f"Bearer {access_token}"},
)
response.text

In [ ]:
def get_namespace_id(namespace_name) -> str:
    response = requests.get(
        f"{CATALOG_URL}/v1/{warehouse_id}/namespaces",
        params={"returnUuids": "true"},  # root namespace -> no parent
        headers={"Authorization": f"Bearer {access_token}"},
    )
    # response is: {'namespaces': [['ns1'], ['ns2'], ['ns3']], 'namespace-uuids': ['ns1 uuid', 'ns2 uuid', 'ns3 uuid']}
    d = response.json()
    return d["namespace-uuids"][d["namespaces"].index([namespace_name])]

In [ ]:
namespace_id = get_namespace_id("customer")  # we'll use `product` namespace as well
namespace_id

In [ ]:
# List namespace permissions

response = requests.get(
    f"{MANAGEMENT_URL}/v1/permissions/namespace/{namespace_id}/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
)
JSON(response.json())

In [ ]:
user_id = get_user_id("Anna Cold")
user_id

In [ ]:
permissions_add = ["select"]

In [ ]:
response = requests.post(
    f"{MANAGEMENT_URL}/v1/permissions/namespace/{namespace_id}/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
    json={
        "writes": [
            {
                "user": user_id,
                "type": p
            }
            for p in permissions_add
        ]
    }
)
response.raise_for_status()

#### 4. Table roles

In [ ]:
def get_table_id(namespace_name: str, table_name: str) -> str:
    response = requests.get(
        url=f"{CATALOG_URL}/v1/{warehouse_id}/namespaces/{namespace_name}/tables/{table_name}",
        headers={"Authorization": f"Bearer {access_token}"},
        params={"pageSize": 100}
    )
    print(response.status_code)
    response.raise_for_status()
    return response.json()["metadata"]["table-uuid"]

In [ ]:
table_id = get_table_id("product", "raw_product")
table_id

In [ ]:
# List table permissions

response = requests.get(
    f"{MANAGEMENT_URL}/v1/permissions/warehouse/{warehouse_id}/table/{table_id}/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
)
print(response.text)

In [ ]:
# Add select permission only on raw_product in product namespace

In [ ]:
user_id = get_user_id("Peter Cold")

In [ ]:
permissions_add = ["select"]

In [ ]:
response = requests.post(
    f"{MANAGEMENT_URL}/v1/permissions/warehouse/{warehouse_id}/table/{table_id}/assignments",
    headers={"Authorization": f"Bearer {access_token}"},
    json={
        "writes": [
            {
                "user": user_id,
                "type": p
            }
            for p in permissions_add
        ]
    }
)
print(response.text)